In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs
import math
import random

from nnfs.datasets import spiral_data
from nnfs.datasets import vertical_data

from layers import Dense, Dropout
from activations import ReLU, SoftMax, Sigmoid
from losses import CategoricalCrossEntropy, Softmax_CategoricalCrossentropy, BinaryCrossentropy
from optimizers import SGD, AdaGrad, RMSProp, Adam

In [3]:
nnfs.init()

## CH 16: Binary Logistic Regression

In [5]:
# Dataset
X, y = spiral_data(samples=100, classes=2)
y = y.reshape(-1,1)

# First Layer
dense1 = Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = ReLU()

# Second Layer
dense2 = Dense(64, 1)
activation2 = Sigmoid()

# Categorical-CrossEntropy with Activation
loss_function = BinaryCrossentropy()

# Optimizer
optimizer = Adam(decay=5e-7)

for epoch in range(10001):
    # Forward Pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Loss Computation
    # Data
    data_loss = loss_function.calculate(activation2.output, y)
    
    # Regularization termr
    regularization_loss = loss_function.regularization_loss(dense1) + loss_function.regularization_loss(dense2)
    
    # Total
    loss = data_loss + regularization_loss
    
    
    # Accuracy
    predictions = (activation2.output > 0.5) * 1
    acc = np.mean(predictions==y)
    
    if not epoch % 1000:
        print(f'epoch: {epoch}, ' +
              f'acc: {acc:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'data_loss: {data_loss:.3f} ' +
              f'reg_loss: {regularization_loss:.3f} ' +
              f'lr: {optimizer.current_learning_rate} ')

    # Backward Pass
    loss_function.backward(activation2.output, y)
    
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Optimize
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.555, loss: 0.693, data_loss: 0.693 reg_loss: 0.000 lr: 0.001 
epoch: 1000, acc: 0.665, loss: 0.622, data_loss: 0.612 reg_loss: 0.010 lr: 0.000999500749375687 
epoch: 2000, acc: 0.820, loss: 0.472, data_loss: 0.422 reg_loss: 0.050 lr: 0.0009990014980027463 
epoch: 3000, acc: 0.880, loss: 0.400, data_loss: 0.334 reg_loss: 0.066 lr: 0.000998502745133672 
epoch: 4000, acc: 0.910, loss: 0.340, data_loss: 0.268 reg_loss: 0.072 lr: 0.0009980044900222008 
epoch: 5000, acc: 0.925, loss: 0.301, data_loss: 0.224 reg_loss: 0.076 lr: 0.000997506731923557 
epoch: 6000, acc: 0.935, loss: 0.276, data_loss: 0.199 reg_loss: 0.077 lr: 0.0009970094700944517 
epoch: 7000, acc: 0.945, loss: 0.257, data_loss: 0.182 reg_loss: 0.075 lr: 0.000996512703793076 
epoch: 8000, acc: 0.945, loss: 0.242, data_loss: 0.171 reg_loss: 0.072 lr: 0.0009960164322790998 
epoch: 9000, acc: 0.945, loss: 0.230, data_loss: 0.162 reg_loss: 0.068 lr: 0.000995520654813666 
epoch: 10000, acc: 0.950, loss: 0.219, data_

In [7]:
# Create test set
X_test, y_test = spiral_data(samples=100, classes=2)
y_test = y_test.reshape(-1,1)

# Evaluate the model on test set
# Forward pass
dense1.forward(X_test)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

loss = loss_function.calculate(activation2.output, y_test)

predictions = (activation2.output > 0.5) * 1
acc = np.mean(predictions==y_test)

print(f'Validation acc: {acc:.3f}, loss: {loss:.3f}')

Validation acc: 0.920, loss: 0.255
